In [19]:
import pandas as pd
import mysql.connector

# Read CSV file into a DataFrame
df = pd.read_csv('summer_movies.csv')
df1 = df.copy()
data = pd.read_csv('summer_movie_genres.csv')
data1 = data.copy()


# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='SummerMovies'
)
cursor = conn.cursor()

# Insert Data into MySQL
for _, row in df1.iterrows():
    cursor.execute(
        """
        INSERT ignore INTO movies (tconst, title_type, primary_title, original_title, year, runtime_minutes, genres, simple_title, average_rating, num_votes)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (row['tconst'], row['title_type'], row['primary_title'], row['original_title'], row['year'], row['runtime_minutes'],
         row['genres'], row['simple_title'], row['average_rating'], row['num_votes'])
    )

for _, row in data1.iterrows():
    cursor.execute(
        """
        INSERT INTO movie_type(tconst,genres)
        VALUES (%s, %s)
        """,
        (row['tconst'],
         row['genres'])
    )


# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()


In [20]:
df1.head(3)
df1.info()
df1.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           905 non-null    object 
 1   title_type       905 non-null    object 
 2   primary_title    905 non-null    object 
 3   original_title   905 non-null    object 
 4   year             904 non-null    float64
 5   runtime_minutes  849 non-null    float64
 6   genres           887 non-null    object 
 7   simple_title     905 non-null    object 
 8   average_rating   905 non-null    float64
 9   num_votes        905 non-null    int64  
dtypes: float64(3), int64(1), object(6)
memory usage: 70.8+ KB


,year,runtime_minutes,average_rating,num_votes
count,904.000000,849.000000,905.000000,905.000000
mean,2001.353982,91.043581,6.273702,2296.285083
std,19.448166,23.854447,1.245414,20630.393488
min,1920.000000,5.000000,2.300000,10.000000
25%,1990.000000,82.000000,5.600000,34.000000
50%,2008.000000,91.000000,6.400000,119.000000
75%,2016.000000,101.000000,7.100000,517.000000
max,2024.000000,237.000000,9.800000,564894.000000


In [21]:
data1.info()
data1.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585 entries, 0 to 1584
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tconst  1585 non-null   object
 1   genres  1567 non-null   object
dtypes: object(2)
memory usage: 24.9+ KB


,tconst,genres
count,1585,1567
unique,905,25
top,tt9842874,Drama
freq,3,489


In [25]:
df1.columns               


Index(['tconst', 'title_type', 'primary_title', 'original_title', 'year',
       'runtime_minutes', 'genres', 'simple_title', 'average_rating',
       'num_votes'],
      dtype='object')

In [24]:
data1.columns

Index(['tconst', 'genres'], dtype='object')

In [ ]:
# Data Exploration:

# df1.head(): View the first few rows of the DataFrame.
# df1.info(): Get a summary of the DataFrame, including data types and non-null counts.
# df1.describe(): Get descriptive statistics for numeric columns.

# Sorting:

# df1.sort_values(by='average_rating', ascending=False): Sort movies by average rating in descending order.
# df1.sort_values(by=['year', 'num_votes'], ascending=[True, False]): Sort by year (ascending) and then by number of votes (descending).
# Handling Missing Data:

# df1.isnull().sum(): Check for missing values in each column.
# df1.dropna(): Drop rows with missing values.
# df1.fillna(0): Replace missing values with 0.
# Adding and Removing Columns:

# df1['new_column'] = df1['average_rating'] * df1['num_votes']: Create a new column based on existing ones.
# df1.drop('genres', axis=1): Remove the genres column.
# Reshaping Data:

# df1.pivot_table(index='year', columns='genres', values='average_rating'): Create a pivot table to see average ratings per genre by year.
# df1.melt(id_vars=['tconst'], value_vars=['year', 'runtime_minutes']): Unpivot selected columns.


In [31]:
# Filtering and Selection:

#  Filter movies released after the year 2000.
df1[df1['year'] > 2000]

#  Select specific columns.
df1[['primary_title', 'year']]


,primary_title,year
0,Midsummer Madness,1920.0
1,A Midsummer Night's Dream,1935.0
2,The Teachers on Summer Vacation,1941.0
3,Summer Storm,1944.0
4,Centennial Summer,1946.0
...,...,...
900,Summer Love,2019.0
901,Let the Summer Never Come Again,2017.0
902,Goodbye Summer,2019.0
903,Summer of Mesa,2020.0


In [32]:
df1.isna().sum()

tconst              0
title_type          0
primary_title       0
original_title      0
year                1
runtime_minutes    56
genres             18
simple_title        0
average_rating      0
num_votes           0
dtype: int64

In [33]:
data1.isna().sum()

tconst     0
genres    18
dtype: int64

In [37]:
df1['genres']=df1['genres'].fillna('unknown')
data1['genres']=data1['genres'].fillna('unknown')

In [38]:
data1.isna().sum()

tconst    0
genres    0
dtype: int64

In [40]:
data1['genres']

0         Drama
1        Comedy
2       Fantasy
3       Romance
4        Comedy
         ...   
1580      Drama
1581    Romance
1582     Comedy
1583      Drama
1584    History
Name: genres, Length: 1585, dtype: object

In [42]:
# Grouping and Aggregation:

# df1.groupby('genres').agg({'average_rating': 'mean', 'num_votes': 'sum'}): Get average ratings and total votes by genre.
df1.groupby('genres').agg({'average_rating':'mean', 'num_votes':'sum'})
# Count the number of movies released each year by genre.
df1.groupby(['year', 'genres']).size()

year    genres                
1920.0  Drama                     1
1928.0  Comedy                    1
1935.0  Comedy,Fantasy,Romance    1
1941.0  Comedy                    1
1944.0  Comedy,Drama,Music        1
                                 ..
2024.0  Drama,History,Western     1
        Drama,Romance             1
        Drama,Thriller            1
        Thriller                  1
        unknown                   2
Length: 646, dtype: int64

In [44]:
# Merging DataFrames:

# merged_df = pd.merge(df1, data1, on='tconst'): Merge movies and movie_type DataFrames on tconst.
merged_df = pd.merge(df1, data1, on ='tconst')
merged_df.head(5)

,tconst,title_type,primary_title,original_title,year,runtime_minutes,genres_x,simple_title,average_rating,num_votes,genres_y
0,tt0011462,movie,Midsummer Madness,Midsummer Madness,1920.0,60.0,Drama,midsummer madness,7.4,19,Drama
1,tt0026714,movie,A Midsummer Night's Dream,A Midsummer Night's Dream,1935.0,133.0,"Comedy,Fantasy,Romance",a midsummer nights dream,6.8,3931,Comedy
2,tt0026714,movie,A Midsummer Night's Dream,A Midsummer Night's Dream,1935.0,133.0,"Comedy,Fantasy,Romance",a midsummer nights dream,6.8,3931,Fantasy
3,tt0026714,movie,A Midsummer Night's Dream,A Midsummer Night's Dream,1935.0,133.0,"Comedy,Fantasy,Romance",a midsummer nights dream,6.8,3931,Romance
4,tt0033864,movie,The Teachers on Summer Vacation,Magistrarna på sommarlov,1941.0,86.0,Comedy,the teachers on summer vacation,5.5,78,Comedy


In [45]:
df1.index

RangeIndex(start=0, stop=905, step=1)

In [46]:
df1.sort_index() # sort by index

,tconst,title_type,primary_title,original_title,year,runtime_minutes,genres,simple_title,average_rating,num_votes
0,tt0011462,movie,Midsummer Madness,Midsummer Madness,1920.0,60.0,Drama,midsummer madness,7.4,19
1,tt0026714,movie,A Midsummer Night's Dream,A Midsummer Night's Dream,1935.0,133.0,"Comedy,Fantasy,Romance",a midsummer nights dream,6.8,3931
2,tt0033864,movie,The Teachers on Summer Vacation,Magistrarna på sommarlov,1941.0,86.0,Comedy,the teachers on summer vacation,5.5,78
3,tt0037325,movie,Summer Storm,Summer Storm,1944.0,106.0,"Crime,Drama,Film-Noir",summer storm,6.6,688
4,tt0038406,movie,Centennial Summer,Centennial Summer,1946.0,102.0,"History,Music,Romance",centennial summer,6.1,431
...,...,...,...,...,...,...,...,...,...,...
900,tt9382076,movie,Summer Love,Summer Love,2019.0,NaN,Romance,summer love,7.5,17
901,tt9430712,movie,Let the Summer Never Come Again,Lass den Sommer nie wieder kommen,2017.0,202.0,Drama,let the summer never come again,6.2,98
902,tt9530420,movie,Goodbye Summer,Goodbye Summer,2019.0,67.0,"Drama,Romance",goodbye summer,5.6,79
903,tt9716208,movie,Summer of Mesa,Summer of Mesa,2020.0,76.0,"Drama,Romance",summer of mesa,5.5,114


In [48]:
df1.sort_values(by='year') # sort by year

,tconst,title_type,primary_title,original_title,year,runtime_minutes,genres,simple_title,average_rating,num_votes
0,tt0011462,movie,Midsummer Madness,Midsummer Madness,1920.0,60.0,Drama,midsummer madness,7.4,19
207,tt0134719,movie,Her Summer Hero,Her Summer Hero,1928.0,60.0,Comedy,her summer hero,4.3,12
1,tt0026714,movie,A Midsummer Night's Dream,A Midsummer Night's Dream,1935.0,133.0,"Comedy,Fantasy,Romance",a midsummer nights dream,6.8,3931
2,tt0033864,movie,The Teachers on Summer Vacation,Magistrarna på sommarlov,1941.0,86.0,Comedy,the teachers on summer vacation,5.5,78
249,tt0221249,movie,Summer Hotel,Hotel de verano,1944.0,95.0,"Comedy,Drama,Music",summer hotel,6.0,13
...,...,...,...,...,...,...,...,...,...,...
641,tt22002112,movie,The Summer of Our Love,Mezhdu nami leto,2024.0,113.0,"Comedy,Drama,Musical",the summer of our love,5.9,42
569,tt17037988,movie,Last Flamingo of the Red Summer Sunset,Last Flamingo of the Red Summer Sunset,2024.0,80.0,"Drama,History,Western",last flamingo of the red summer sunset,8.7,10
719,tt30949231,movie,My Summer with Irene,Quell'estate con Irène,2024.0,90.0,Drama,my summer with irene,6.2,39
623,tt21151850,movie,Edge of Summer,Edge of Summer,2024.0,98.0,"Drama,Thriller",edge of summer,8.1,17


In [53]:
df1.head(3)# access by label

,tconst,title_type,primary_title,original_title,year,runtime_minutes,genres,simple_title,average_rating,num_votes
0,tt0011462,movie,Midsummer Madness,Midsummer Madness,1920.0,60.0,Drama,midsummer madness,7.4,19
1,tt0026714,movie,A Midsummer Night's Dream,A Midsummer Night's Dream,1935.0,133.0,"Comedy,Fantasy,Romance",a midsummer nights dream,6.8,3931
2,tt0033864,movie,The Teachers on Summer Vacation,Magistrarna på sommarlov,1941.0,86.0,Comedy,the teachers on summer vacation,5.5,78


In [56]:
row = df1.loc[0]
row

tconst                     tt0011462
title_type                     movie
primary_title      Midsummer Madness
original_title     Midsummer Madness
year                          1920.0
runtime_minutes                 60.0
genres                         Drama
simple_title       midsummer madness
average_rating                   7.4
num_votes                         19
Name: 0, dtype: object

In [58]:
row = df1.loc[0:7, 'genres']
row

0                     Drama
1    Comedy,Fantasy,Romance
2                    Comedy
3     Crime,Drama,Film-Noir
4     History,Music,Romance
5             Drama,Fantasy
6                    Comedy
7                   Musical
Name: genres, dtype: object

In [60]:
row = df1.iloc[0, 6]
row

'Drama'

In [61]:
row = df.iloc[[1,2,4], [0,2]]
row

,tconst,primary_title
1,tt0026714,A Midsummer Night's Dream
2,tt0033864,The Teachers on Summer Vacation
4,tt0038406,Centennial Summer


In [64]:
# concat
frames = [df1, data1]
result = pd.concat(frames)
result

,tconst,title_type,primary_title,original_title,year,runtime_minutes,genres,simple_title,average_rating,num_votes
0,tt0011462,movie,Midsummer Madness,Midsummer Madness,1920.0,60.0,Drama,midsummer madness,7.4,19.0
1,tt0026714,movie,A Midsummer Night's Dream,A Midsummer Night's Dream,1935.0,133.0,"Comedy,Fantasy,Romance",a midsummer nights dream,6.8,3931.0
2,tt0033864,movie,The Teachers on Summer Vacation,Magistrarna på sommarlov,1941.0,86.0,Comedy,the teachers on summer vacation,5.5,78.0
3,tt0037325,movie,Summer Storm,Summer Storm,1944.0,106.0,"Crime,Drama,Film-Noir",summer storm,6.6,688.0
4,tt0038406,movie,Centennial Summer,Centennial Summer,1946.0,102.0,"History,Music,Romance",centennial summer,6.1,431.0
...,...,...,...,...,...,...,...,...,...,...
1580,tt9716208,NaN,NaN,NaN,NaN,NaN,Drama,NaN,NaN,NaN
1581,tt9716208,NaN,NaN,NaN,NaN,NaN,Romance,NaN,NaN,NaN
1582,tt9842874,NaN,NaN,NaN,NaN,NaN,Comedy,NaN,NaN,NaN
1583,tt9842874,NaN,NaN,NaN,NaN,NaN,Drama,NaN,NaN,NaN


Index(['tconst', 'title_type', 'primary_title', 'original_title', 'year',
       'runtime_minutes', 'genres', 'simple_title', 'average_rating',
       'num_votes'],
      dtype='object')

In [ ]:
df